**Operations Research in Action &#x25aa; Fall 2024**

# Project 2 &ndash; Model and Results &ndash; Part 3

Let's start with some data wrangling to prepare the data for use in an optimization model.

__Read in the player ratings data.__

__Read in the estimated coefficients that we wrote to a CSV file at the end of Part 2. When reading in the data, set the column name for the coefficients to `apm`.__

__Merge these two DataFrames by player.__

Hint: the APM for Chile_p12 will be NA, so you'll have to fill in this one manually. To replace all values of NA in a DataFrame with 0, you can apply `.fillna(0)` to the DataFrame.

Now, let's find the lineup for Canada with the maximum total APM.

__On a sheet of paper, write down your optimization model.__

Don't forget to define all the components:

- Sets
- Parameters
- Decision variables
- Objective function
- Constraints

__Now that you've written down your model, implement it using Pyomo.__

__Interpret the optimal solution: what is the optimal lineup?__

__Find the All-Star Team: find the lineup with the maximum total APM across *all* teams.__